This project takes a csv file containg the columns first name, last name, street address, town, and state, and outputs an Excel file containing the columns index, name, address, closest food bank, and driving distance.  

I have attached sample data using fake names and addresses for testing purposes. You can download it using this link:https://docs.google.com/spreadsheets/d/147-BhKixbH5aHTdbF3ON6MfaNd5jPiviDojE15Yrt68/edit?usp=sharing. You will have to get a google maps API key and upload it in a txt file named key.txt if you would like to test out the program for yourself.

In [ ]:
import json
import requests
import pandas as pd
import openpyxl
from google.colab import files
#import key.txt and data.csv:
uploaded = files.upload()


In [ ]:

#Google Maps API key

with open("key.txt", mode = "r") as file:
  key = file.read().strip(" ")

#Empty DataFrame to which new data will be added
df = pd.DataFrame(columns=["Name", "Address", "Closest Pantry", "Driving Distance"])

#Function that accepts adresses as arguments and returns the two destination addresses closest to the origin
def get_shortest(origin, destinations):

    destinations = "|".join(destinations)
    dmatrix_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&key={key}&origins={origin}&destinations={destinations}"
    results = requests.get(dmatrix_url).json()

    if results["status"] == "OK":
        tempdict = {}

        tempdict = {address: float((results["rows"][0]["elements"][i]["distance"]["text"]).strip("mi ")) for i, address in enumerate(results["destination_addresses"])}

        tempdict = sorted(tempdict.items(), key=lambda x: x[1])
        closest = tempdict[0]
        secondclosest = tempdict[1]
        return closest, secondclosest, results["status"]
    else:
        return results["status"], results["status"], results["status"]


# populating dataframe with original data

with open("data.csv", mode="r") as file:
    tempdf = pd.read_csv(file)



for i, row in enumerate(tempdf.itertuples()):
    print(f"row {i}:{row[1]} {row[2]}")

    alt_pantries = {

        "boston": {
            "ABCD Allston/Brighton Neighborhood Opportunity Center": "640 Washington St, Brighton, MA 02135, USA",
            "ABCD Parker Hill Fenway Neighborhood Service Center": "714 Parker St, Boston, MA 02120, USA",
            "ABCD Roxbury/ N. Dorchester Neighborhood Opportunity Center": "565 Warren St, Boston, MA 02121, USA",
            "Resurrection Lutheran Church": "94 Warren St, Boston, MA 02119, USA",
            "ABCD South End Neighborhood Service Center": "554 Columbus Ave, Boston, MA 02118, USA",
            "American Red Cross Boston Food Pantry": "1033 Massachusetts Ave, Boston, MA 02118, USA",
            "Church of the Covenant": "67 Newbury St, Boston, MA 02116, USA",
            "Catholic Charities Yawkey Center": "185 Columbia Rd, Boston, MA 02121, USA",
            "Catholic Charities - El Centro": "1400 Washington St, Boston, MA 02118, USA"
        },

        "allston/brighton": {
            "Allston Brighton Food Pantry": "404 Washington St, Boston, MA 02135, USA",
            "Allston Brighton Area Planning Action Committee": "406 Cambridge St, Boston, MA 02134, USA"
        },

        "all": {
            "Fenway Cares": "1282 Boylston St, Boston, MA 02215, USA",
            "ICNA New England": "100 Malcolm X Blvd, Boston, MA 02119, USA",
            "Sojourner House": "85 Rockland St, Boston, MA 02119, USA",
            "Haley House": "23 Dartmouth St, Boston, MA 02116, USA",
            "The Salvation Army - South End Corps Community Center": "1500 Washington St, Boston, MA 02118, USA"
        },

        "West Roxbury, Roslindale, Hyde Park, Jamaica Plain, Mattapan": {
            "Rose’s Bounty": "77 Stratford St, West Roxbury, MA 02132, USA"
        }
    }

    if row[4].strip(" ").lower() == "brighton" or row[4].strip(" ").lower() == "allston":

        destinations = list(alt_pantries["allston/brighton"].values()) + list(alt_pantries["boston"].values()) + list(alt_pantries["all"].values())

        dict = alt_pantries["allston/brighton"]
        dict.update(alt_pantries["boston"])
        dict.update(alt_pantries["all"])

        result = get_shortest(f"{row[3]} {row[4]} {row[5]}", destinations)
        if result[2] != "OK":
            df = df._append({"Name": f"{row[1]} {row[2]}",
                             "Address": f"{row[3]} {row[4]} {row[5]}",
                             "Closest Pantry": result[2],
                             "Driving Distance": result[2]}, ignore_index=True)
        else:
            df = df._append({"Name": f"{row[1]} {row[2]}",
                             "Address": f"{row[3]} {row[4]} {row[5]}",
                             "Closest Pantries": f"{list(dict.keys())[list(dict.values()).index(result[0][0])]}, {result[0][0]}, {list(dict.keys())[list(dict.values()).index(result[1][0])]}, {result[1][0]}",
                             "Driving Distances": f"{result[0][1]}, {result[1][1]}"}, ignore_index=True)

    elif row[4].strip(" ").lower() == "boston":

        destinations = list(alt_pantries["boston"].values()) + list(alt_pantries["all"].values())

        dict = alt_pantries["boston"]
        dict.update(alt_pantries["all"])

        result = get_shortest(f"{row[3]} {row[4]} {row[5]}", destinations)
        if result[2] != "OK":
            df = df._append({"Name": f"{row[1]} {row[2]}",
                             "Address": f"{row[3]} {row[4]} {row[5]}",
                             "Closest Pantry": result[2],
                             "Driving Distance": result[2]}, ignore_index=True)
        else:
            df = df._append({"Name": f"{row[1]} {row[2]}",
                             "Address": f"{row[3]} {row[4]} {row[5]}",
                             "Closest Pantries": f"{list(dict.keys())[list(dict.values()).index(result[0][0])]}, {result[0][0]}, {list(dict.keys())[list(dict.values()).index(result[1][0])]}, {result[1][0]}",
                             "Driving Distances": f"{result[0][1]}, {result[1][1]}"}, ignore_index=True)
    elif row[4].strip(" ").lower() == "jamaica plain":

        destinations = list(alt_pantries["boston"].values()) + list(alt_pantries["all"].values()) + list(alt_pantries["West Roxbury, Roslindale, Hyde Park, Jamaica Plain, Mattapan"].values())

        dict = alt_pantries["boston"]
        dict.update(alt_pantries["all"])
        dict.update(alt_pantries["West Roxbury, Roslindale, Hyde Park, Jamaica Plain, Mattapan"])

        result = get_shortest(f"{row[3]} {row[4]} {row[5]}", destinations)
        if result[2] != "OK":
            df = df._append({"Name": f"{row[1]} {row[2]}",
                             "Address": f"{row[3]} {row[4]} {row[5]}",
                             "Closest Pantry": result[2],
                             "Driving Distance": result[2]}, ignore_index=True)
        else:
            df = df._append({"Name": f"{row[1]} {row[2]}",
                             "Address": f"{row[3]} {row[4]} {row[5]}",
                             "Closest Pantries": f"{list(dict.keys())[list(dict.values()).index(result[0][0])]}, {result[0][0]}, {list(dict.keys())[list(dict.values()).index(result[1][0])]}, {result[1][0]}",
                             "Driving Distances": f"{result[0][1]}, {result[1][1]}"}, ignore_index=True)

    else:

        destinations = list(alt_pantries["all"].values())

        dict = alt_pantries["all"]

        result = get_shortest(f"{row[3]} {row[4]} {row[5]}", destinations)
        if result[2] != "OK":
            df = df._append({"Name": f"{row[1]} {row[2]}",
                             "Address": f"{row[3]} {row[4]} {row[5]}",
                             "Closest Pantry": result[2],
                             "Driving Distance": result[2]}, ignore_index=True)
        else:
            df = df._append({"Name": f"{row[1]} {row[2]}",
                             "Address": f"{row[3]} {row[4]} {row[5]}",
                             "Closest Pantries": f"{list(dict.keys())[list(dict.values()).index(result[0][0])]}, {result[0][0]}, {list(dict.keys())[list(dict.values()).index(result[1][0])]}, {result[1][0]}",
                             "Driving Distances": f"{result[0][1]}, {result[1][1]}"}, ignore_index=True)

with pd.ExcelWriter("Closest_Pantries.xlsx") as writer:
    df.to_excel(writer, sheet_name="Closest Pantries")


row 0:lorine  ellis
row 1:richard  organ
row 2:melissa wolf
row 3:angie  pruitt
row 4:pierre rogers
